In [18]:
'''
# 三章 ニューラルネットワーク
- 適切な重みパラメータをデータから自動で学習できるというのがニューラルネットワーク(NN)の性質の一つ
- ニューラルネットワークの概要図(P40)
    - 入力層 -> 中間層（隠れ層） -> 出力層
    - 構造だけみるとパーセプトロンと変わらない. 
- 一旦ここでパーセプトロンの復習と別表現方法を導入
    - 入出力を関数として表現する。
    - y = h(b + w1x1 + w2x2)
    - この時のhを活性化関数と呼ぶことにする
        - hは発火する閾値をコントロールしている
    - この時、従来のしきは更に2つに分割できる
        - a = b + w1x1 + w2x2
- 二章で見たような、とある閾値を境に発火するか否かを切り替える関数をステップ関数、階段関数と呼ぶ
    - よって、パーセプトロンでは活性化関数にステップ関数を利用しているといえることができる
- 活性化関数にステップ関数以外を導入することで、NNの世界にすすめる!

## シグモイド関数
- NNでよく用いられている活性化関数の一つ
    - h(x) = 1 / (1 + exp(-x))
- パーセプトロンとNNの違いは活性化関数の違いくらい。多層につながる構造や信号の伝達方法は同じ
'''

# step関数の実装
def step_function(x):
    return 1 if x > 0 else 0

# step関数をNumPyArrayを引数にもてるように変更
def step_function(x):
    y = x > 0
    return y.astype(np.int)


'''
- [x] why numpyの配列を引数に持つような実装にする必要があるのか...
    - 一括してステップ関数に複数の入力値を渡していた
- np.ndarrayに対して不等号を与えると、map(lambda item:item > 0, x)したのと同じ結果をもつ配列が返される
    - np.ndarrayはNumPyで最も重要なクラス。 N-d Array すなわち，N次元配列を扱うためのクラス
- npのastypeメソッドは型変換を行なう
    - booleanをintに変換すると、trueが1に、falseが0に変換される
- つまり、この関数は複数の入力値を一括でステップ関数に渡す処理を行っている

## step関数のグラフ
- グラフ化にはmatplotlibを利用する
- np.array(np.ndarray, dtype=None)
    - 引数のnp.ndarray or arrayをnp.ndarrayに変換する。
    代に引数にdtypeを指定するとそれに変換してくれる
'''

import matplotlib.pylab as plt
import numpy as np

def step_function(x):
    return np.array(x > 0, dtype=np.int) # 前述と同じ実装


x = np.arange(-5.0, 5.0, 0.1) # -5.0 ~ 5.0までの区間を0.1刻みで配列生成
y = step_function(x)

plt.plot(x, y)
ｐlt.ylim(-0.1, 1.1) # y軸の範囲を指定
# plt.show()


'''
## シグモイド関数の実装
'''

def sigmoid(x):
    return 1/(1 + np.exp(-x))
    # このxにnp.ndarrayを放り込んでも適切にmapで処理してくれる
    # numpyのブロードキャストという機能による
    # np.expがnp.ndarray


y = sigmoid(x)

plt.plot(x, y)
ｐlt.ylim(-0.1, 1.1) # y軸の範囲を指定
# plt.pause(1)

'''
- 相違点
    - グラフの滑らかさ
    - つまりsigmoidは0/1ではなく、0-1の間の実数を返す
- 共通点
    - 出力を0-1に押し込める
    - 入力が大きくなるに連れて出力も大きくなる
    - ともに非線形関数
- NNでは、活性化関数に**非線形関数を用いる必要がある**
    - 線形関数をもちいると、線形関数ではNNの層を深くすることが出来ないため
- その理由の簡単な説明
    - h(x) = cx(c: 定数)とした時、y(x) = h(h(h(x)))と3層にしたとしても、それはh2(x) = c^3 * xと同義になってしまう
    - つまり、多層にする利点を活かすことが出来ない
    - [ ] 線形の活性化関数だと多層にしても1層の場合と同じになってしまうということは分かったが、多層の利点がわからないのでいまいちピンとこない

## ReLU関数
- sigmoid関数は古くからNNにおいて用いられてきた
- 最近ではRectified Linear Unit関数が主に用いられる
    - 入力が0を超えていれば、その入力をそのまま出力する
'''

def refu(x):
    return np.maximum(0, x)

'''
## 多次元配列の計算
- 一旦ここで、np.ndarrayの配列計算を学ぶ
'''

A = np.array([1, 2, 3, 4])
# print(A)        # [1, 2, 3, 4]
np.ndim(A)  # 1次元
A.shape       # (4, ) ... 1次元目の要素数が4つという意味
A.shape[0] # 4

B = np.array([[1,2], [3, 4], [5, 6]])
# print(B)        
np.ndim(B)  # 2次元
B.shape       # (3, 2) ... 3 * 2の行列

# 行列の内積 ... いわゆる行列の席を返す
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

np.dot(A, B) # A・B
'''
array([[19, 22],
            [43, 50]])
'''

# これとは別
A * B
'''
array([[ 5, 12],
           [21, 32]])
'''

'''
## NNの実装
- まず、バイアスと活性化関数は無視し、重みだけがあるとするものを実装する場合。
'''

X = np.array([1, 2])
W = np.array([[1, 3, 5], [2, 4, 6]])
Y = np.dot(X, W)
print(Y) # [ 5 11 17]

'''
- 実戦的なNNの実装
    - 3層のNNを実装してみる
        - ニューロンの数: 2 -> 3 -> 2 -> 2のようなモデル
    - P60
- 2層目から出力層への活性化関数だけを変えている
    - 入力をそのまま出力する関数を恒等関数という
    - [ ] why 恒等関数を利用するのか
    - 出力層で利用する活性化関数は、解く問題の性質によって変える
        - 回帰問題 => 恒等関数
        - 2クラス分類 => シグモイド関数
        - 他クラス分類 => ソフトマックス関数
'''

# 0層目から生まれた1層目を表現すると、
# A1 = XW1 + B1となるので、

X = np.array([1.0, 0.5])
W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
B1 = np.array([0.1, 0.2, 0.3])

A1 = np.dot(X, W1) + B1

# A1を活性化関数にとうしたものをZ1とする
Z1 = sigmoid(A1)

# 2層目の表現
W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
B2 = np.array([0.1, 0.2])
A2 = np.dot(A1, W2) + B2
Z2 = sigmoid(A2)


# 2層目から出力層への活性化関数だけを変える

def identify_function(x):
    return x

W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
B3 = np.array([0.1, 0.2])

A3 = np.dot(A2, W3)
Y = identify_function(A3)
print(Y)



# ここまでを関数に落とし込むと...

def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['W3'] = np.array([[0.1, 0.3], [0.2, 0.4]])
    network['B1'] = np.array([0.1, 0.2, 0.3])
    network['B2'] = np.array([0.1, 0.2])
    network['B3'] = np.array([0.1, 0.2])
    return network

def pass_layer(x, w, b, active_func):
    tmp = np.dot(x, w) + b
    return active_func(tmp)


def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    B1, B2, B3 = network['B1'], network['B2'], network['B3']
    a1 = pass_layer(x, W1, B1, sigmoid)
    a2 = pass_layer(a1, W2, B2, sigmoid)
    y = pass_layer(a2, W3, B3, identify_function)
    return y
    
nw = init_network();
x = np.array([1.0, 0.5])
forward(nw, x) # array([ 0.31682708,  0.69627909])

'''
## 出力層の設計
- NNは分類問題と回帰問題の両方に用いることができる
- ただし、どちらに用いるかで出力層の活性化関数を変更する必要がある
    - 回帰問題 => 恒等関数
    - 分類問題 => ソフトマックス関数
    - [ ] whyこういう使い分けが生まれる
- なお、機械学習の問題は分類と回帰に大別できる
- 恒等関数
    - 入力と出力が一致する関数
- ソフトマックス関数
    - yk = exp(ak) / ∑(exp(ai)) ... i, kは添字
    - 出力が全ての入力値から影響を受ける特質がある
    - また、値が0-1に収まること、softmaxの返す配列の総和は常に1になることから、確率のようにみなすことができる
    - [x] 何故にsoftmaxという名前なのか
        -  x の各成分の中で xi がダントツで大きい → yi はほぼ 1 で y の他の成分はほぼ 0 になるような関数
        - マックス関数（一番大きい成分を 11 にして，それ以外のものは 00 にする関数をこう呼ぶことにする）をソフトにしたという感じ
        - [ソフトマックス関数 | 高校数学の美しい物語](http://mathtrain.jp/softmax)
'''

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a) # array
sum_exp_a = np.sum(exp_a)
y = exp_a / sum_exp_a

def softmax(a):
    exp_a = np.exp(a) # array
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

'''
- しかし、このmethodだとaのいち要素が大きい時、例えば10000などの場合、e^10000を計算することになり、数が跳ね上がる
- 結果、オーバーフローが起こりいる
- そこで、このメソッドを次の特性を用いて修正する
    - exp(ak) / ∑(exp(ai)) = (exp(ak) + C) / ∑(exp(ai) + C) ... p69
    - Cに入力信号の中で最もおおきな値を用いることで数の膨れ上がりをある程度抑えることができる
'''

def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

'''
- softmax関数では、値が0-1に収まること、softmaxの返す配列の総和は常に1になることから、確率のようにみなすことができる
- 出力内容を確率とみなすと、[0.018, 0.245, 0.736]は「74%の確率で二番目のクラスである」と解釈することができる
- NNのクラス分類では、一般的に出力の一番大きいニューロンに相当するクラスだけを認識結果とする
- また、softmaxでは、入力と出力の序列関係を変えることはないので、NNで分類をする際、出力層のソフトマックス層を省略する事ができる
- [ ] なんとなく、softmaxは特徴を際立たせるための活性化関数のように見えるが、その解釈でいいのだろうか

## 出力層のニューロンの数
- 出力層のニューロンの数はとくべき問題に応じて適宜決める
- クラス分類を行なう問題では、例えば数字を0-9のどれか判定する場合であれば10クラス分類なのでニューロンも10個用意する必要がある

## 手書き数字認識
- ここでは一旦「学習」は終わっているとして、学習済みのパラメータを使って、推論処理だけを行なう
    - [ ] この推論処理をNNの順方向伝播と言うらしいが、なにを「この」と指しているかわからない
- MNISTデータセット
    - 機械学習の分野で最も有名なデータセットの一つ. よく使われる
    - 訓練画像が6万枚, テスト画像が1万枚用意されている
    - 一つ一つは28 * 28のグレー画像で、書くピクセルは0 - 255の値を取る
- NNの推論処理を実装する
    - 28 * 28のピクセルなので、入力層は784個のニューロンを持ち、出力層は10個のニューロンを持つ
'''

# メモがある場所からoreillyのrepoを見えるようにする
import sys
sys.path.append('deep-learning-from-scratch')
from dataset.mnist import load_mnist

import pickle

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_labeｌ=False)
    return x_test, t_test

def init_network():
    # pythonにはpickleという、プログラmの実行中オブジェクトをファイルとして保存する機能がある
    # rubyでいうserializeしたobjectみたいなもの？
    with open('./deep-learning-from-scratch/ch03/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    '''
    with open("...") as f:
        print(f.read())
    は、以下と同じ意味。indent抜けた際に自動的にcloseしてくれる
    f = open("...")
    print(f.read())
    f.close()
    '''    
        
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    B1, B2, B3 = network['b1'], network['b2'], network['b3']
    
    z1 = pass_layer(x, W1, B1, sigmoid)
    z2 = pass_layer(z1, W2, B2, sigmoid)
    y = pass_layer(z2, W3, B3, softmax)
    return y

x, t = get_data()
nw = init_network()

accracy_cnt = 0
for i in range(len(x)):
    y = predict(nw, x[i])
    p = np.argmax(y) # yの中で最大の要素 = 確率が最も高いもののindexを取得
    if p == t[i]:
        accracy_cnt += 1
        
print('Accracy: ' + str(float(accracy_cnt / len(x))))
# Accracy: 0.9352

'''
- load_mnist関数でnormalize=Trueにすると、画像の各ピクセルを255で割り、データの値が0 - 1に収まるように変換する
- このようなデータを歩き待った範囲に変換する処理を正規化という
- NNの入力データに対して、何らかの決まった変換を行なうことを前処理という
    - 識別性能の向上やスピードアップのために行なう
    - 通常は標準偏差と平均をもちいて0を中心に分布するような正規化を行なうことがおおい
'''

W1, W2, W3 = nw['W1'],  nw['W2'], nw['W3']
print(x.shape, W1.shape, W2.shape, W3.shape)
# (10000, 784) (784, 50) (50, 100) (100, 10)  -> 10 => 形状を確認。行列の内積を行える形になっている

'''
- 画像のバッチ(まとめて)処理をするにはどうするか
    - pythonの工夫を普通に行なう
- [ ] (28 * 28)個のニューロンからなる入力内容が最終的に0-9のintに収まっていく過程で、隠れそうに当たる層は一体何をしているのか
    - 正規化をしているので、1つのピクセルの実体は0-1の間に収まる実数である
    - それに重みをつけて50個のニューロンにしたり、100個のニューロンにしたりしている
    - その変換の過程でなぜ分類がされていくのか？
    - たとえば、中心点にあたるピクセルの入力値が1 = 黒だったとして、最終的に1につながる重み付けならwが大きく、0に繋がる重み付けならwが小さくなるということ？
'''

x, t = get_data()
nw = init_network()

batch_size = 100
accracy_cnt = 0

for i in range(0, len(x), batch_size): # 第三引数でstepを設定できる
    x_batch = x[i:i+batch_size]
    y_batch = predict(nw, x_ batch) # いま、predictはブロードキャストに対応している
    p = np.argmax(y_batch, axis=1) # 1次元目を基準に最大要素のindexを返す
    accracy_cnt += np.sum(p == t[i:i+batch_size]) 
    # np.ndarray同時の比較は、同じ位置の要素同士を比較したTrue/False配列を返す
    # さらにnp.sumでtrueとなるものだけを数えている

[ 5 11 17]
[ 0.326  0.712]
Accracy: 0.9352
(10000, 784) (784, 50) (50, 100) (100, 10)
